In [31]:
import os
import pandas as pd
import json
from openai import AzureOpenAI
import time

# Initialize the AzureOpenAI client with your API key and endpoint from environment variables
client = AzureOpenAI(
    api_key="Enter your key",
    api_version="2023-05-15",
    azure_endpoint="Enter your endpoint"
)

# Define the function to process a support ticket with Azure OpenAI
def process_ticket_with_azure(ticket_text):
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=[
            {"role": "system", "content":"""
You are a technical assistant processing support tickets. Your tasks are as follows:
1. Classify the ticket into 'Technical Issues', 'Hardware Issues', or 'Data Recovery'.
2. Create relevant tags for the ticket.
3. Assign a 'High' or 'Normal' priority level based on urgency.
4. Provide an estimated time for issue resolution (ETA).
Ensure the output is in JSON format with keys "Category", "Tags", "Priority", "ETA".
Example: {"Category": 'Technical Issues', "Tags": ['Internet', 'Connectivity'], "Priority": 'High', "ETA": '2 hours'}
"""},
            {"role": "user", "content": ticket_text}
        ]
    )
    #print(response)
    
    output_json_str = response.choices[0].message.content
    #print("Response content:", output_json_str)
    
    try:
        output_data = json.loads(output_json_str)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        output_data = {}

    # Provide default values if keys are missing
    category = output_data.get("Category", "Unknown")
    tags = output_data.get("Tags", ["Unknown"])
    priority = output_data.get("Priority", "Unknown")
    eta = output_data.get("ETA", "Unknown")

    return {
        "Category": category,
        "Tags": ", ".join(tags),
        "Priority": priority,
        "ETA": eta
    }

# Read the CSV file from the local machine
df = pd.read_csv(r"C:\Users\sbonagiri\OneDrive - Microsoft\Desktop\aspen\Support_ticket_text_data_mid_term.csv")

start_time = time.time()

# Process each support ticket
for index, row in df.iterrows():
    processed_data = process_ticket_with_azure(row['support_ticket_text'])
    df.at[index, 'Category'] = processed_data['Category']
    df.at[index, 'Tags'] = processed_data['Tags']
    df.at[index, 'Priority'] = processed_data['Priority']
    df.at[index, 'ETA'] = processed_data['ETA']

end_time = time.time()

# Calculate duration
duration = end_time - start_time

# Print the duration
print(f"Script took {duration} seconds to run.")    
#print("Top 5 rows of the processed DataFrame:")
print(df.head())

# Save the updated DataFrame to a new CSV file
output_file_path = r"C:\Users\sbonagiri\OneDrive - Microsoft\Desktop\aspen\Support_ticket_text_data_mid_term_updated_final1.csv"
df.to_csv(output_file_path, index=False)

print(f"Processed data saved to {output_file_path}")


Script took 10.775261878967285 seconds to run.
  support_tick_id                                support_ticket_text  \
0      ST2023-006  My internet connection has significantly slowe...   
1      ST2023-007  Urgent help required! My laptop refuses to sta...   
2      ST2023-008  I've accidentally deleted essential work docum...   
3      ST2023-009  Despite being in close proximity to my Wi-Fi r...   
4      ST2023-010  My smartphone battery is draining rapidly, eve...   

           Category                            Tags Priority      ETA  
0  Technical Issues          Internet, Connectivity     High  2 hours  
1   Hardware Issues         Laptop, Startup Failure     High   1 hour  
2     Data Recovery  Accidental deletion, Data loss     High  4 hours  
3  Technical Issues          Wi-Fi, Signal Strength     High  2 hours  
4   Hardware Issues               Battery, Draining     High    1 day  
Processed data saved to C:\Users\sbonagiri\OneDrive - Microsoft\Desktop\aspen\Support_ti